In [ ]:
import jax.numpy as np
import pandas as pd
import itertools as iter
import warnings
import pickle
import random

warnings.filterwarnings("ignore")
def new_dataframe():
    colNames = []
    for i in range(1, 13):
        colNames.append(f"coeff{i}")

    colNames.append("speaker_count")
    # colNames.append("utter_count")
    # colNames.append("frame_count")

    df = pd.DataFrame(columns=colNames)
    return colNames, df

colNames, df = new_dataframe()
print("Created Dataframe")

WINDOW_SIZE = 5

In [146]:
bounds = [i for i in iter.accumulate([30, 30, 30, 30, 30, 30, 30, 30, 30])]
print(bounds)

speak_count = 0
utter_count = 0
frame_count = -1

train_file = open("./data/ae.train", "r")
print("Loaded data")

for line in train_file.readlines():
    if line == "\n":
        utter_count +=1
        frame_count = -1
        speak_count = (utter_count//30)

    else:
        frame_count += 1

        row = line[:-2].split(" ")
        row.append(speak_count)
        row = np.asarray(row).reshape(1, 13)
        # row.append(utter_count)
        # row.append(frame_count)

        #print(row)

        utter_idx = utter_count - bounds[speak_count - 1] if speak_count > 0 else utter_count
        # print(utter_idx)
        index = pd.MultiIndex.from_tuples([(int(speak_count), int(utter_idx), int(frame_count))], names=["speak", "utter", "frame"])
        row = pd.DataFrame(row, index=index, columns=colNames)

        #print(row)
        df = pd.concat([df, row])

print("Loaded dataframe")
train_file.close()

print(df.shape)
print(df)

[30, 60, 90, 120, 150, 180, 210, 240, 270]
Loaded data
Loaded dataframe
(4274, 13)
              coeff1     coeff2    coeff3     coeff4     coeff5     coeff6  \
(0, 0, 0)   1.860936  -0.207383  0.261557  -0.214562  -0.171253  -0.118167   
(0, 0, 1)   1.891651  -0.193249  0.235363  -0.249118  -0.112890  -0.112238   
(0, 0, 2)   1.939205  -0.239664  0.258561  -0.291458  -0.041053  -0.102034   
(0, 0, 3)   1.717517  -0.218572  0.217119  -0.228186  -0.018608  -0.137624   
(0, 0, 4)   1.741191  -0.279891  0.196583  -0.236377  -0.032012  -0.090612   
...              ...        ...       ...        ...        ...        ...   
(8, 29, 4)  1.141889  -0.485815  0.288329  -0.687604   0.301889  -0.203278   
(8, 29, 5)  1.162979  -0.340980  0.290894  -0.553881   0.324858  -0.250906   
(8, 29, 6)  1.224771  -0.371558  0.335834  -0.547974   0.384870  -0.203234   
(8, 29, 7)  1.030606  -0.309280  0.311761  -0.466693   0.453942  -0.260364   
(8, 29, 8)  0.966744  -0.323093  0.305467  -0.419457   0.51

In [147]:
ran = [31, 35, 88, 44, 29, 24, 40, 50, 29]
bounds = [i for i in iter.accumulate([31, 35, 88, 44, 29, 24, 40, 50, 29])]
print(bounds)

res = {}

speak_count = 0
utter_count = 0
frame_count = -1

test_file = open("./data/ae.test", "r")
print("Loaded test data")

for line in test_file.readlines():
    if line == "\n":
        utter_count +=1
        frame_count = -1

        if utter_count in bounds:
            speak_count = bounds.index(utter_count) + 1

        res[speak_count] = res.get(speak_count, 0) + 1

    else:
        frame_count += 1

        row = line[:-2].split(" ")
        row.append(speak_count)
        row = np.asarray(row).reshape(1, 13)
        # row.append(utter_count)
        # row.append(frame_count)

        utter_idx = utter_count - bounds[speak_count - 1] if speak_count > 0 else utter_count
        # print(utter_idx)
        index = pd.MultiIndex.from_tuples([(int(speak_count), int(30 + utter_idx), int(frame_count))], names=["speak", "utter", "frame"])
        row = pd.DataFrame(row, index=index, columns=colNames)

        #print(row)
        df = pd.concat([df, row])

print("Loaded dataframe")
test_file.close()
for idx in range(len(ran)):
    print(f"{idx}: {ran[idx]}, {res[idx]}")

[31, 66, 154, 198, 227, 251, 291, 341, 370]
Loaded test data
Loaded dataframe
0: 31, 30
1: 35, 35
2: 88, 88
3: 44, 44
4: 29, 29
5: 24, 24
6: 40, 40
7: 50, 50
8: 29, 29


In [148]:
df

,coeff1,coeff2,coeff3,coeff4,coeff5,coeff6,coeff7,coeff8,coeff9,coeff10,coeff11,coeff12,speaker_count
"(0, 0, 0)",1.860936,-0.207383,0.261557,-0.214562,-0.171253,-0.118167,-0.277557,0.025668,0.126701,-0.306756,-0.213076,0.088728,0
"(0, 0, 1)",1.891651,-0.193249,0.235363,-0.249118,-0.112890,-0.112238,-0.311997,-0.027122,0.171457,-0.289431,-0.247722,0.093011,0
"(0, 0, 2)",1.939205,-0.239664,0.258561,-0.291458,-0.041053,-0.102034,-0.383300,0.019013,0.169510,-0.314894,-0.227908,0.074638,0
"(0, 0, 3)",1.717517,-0.218572,0.217119,-0.228186,-0.018608,-0.137624,-0.403318,-0.009643,0.164607,-0.323267,-0.210105,0.098098,0
"(0, 0, 4)",1.741191,-0.279891,0.196583,-0.236377,-0.032012,-0.090612,-0.363134,-0.012571,0.124298,-0.351171,-0.216545,0.113899,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(8, 58, 6)",1.216506,-0.424432,-0.034349,-0.294238,0.166270,-0.397758,0.091827,0.023178,-0.191598,-0.111245,-0.011547,0.425088,8
"(8, 58, 7)",1.214579,-0.399925,-0.127891,-0.239583,0.221423,-0.374714,0.046216,0.051754,-0.197824,-0.099159,-0.018177,0.359986,8
"(8, 58, 8)",1.170031,-0.306025,-0.145534,-0.195854,0.276371,-0.346794,0.042325,0.031668,-0.181205,-0.077962,-0.024079,0.305890,8
"(8, 58, 9)",1.118108,-0.258605,-0.103982,-0.231256,0.344866,-0.223807,-0.014415,-0.046215,-0.219385,0.013357,-0.010018,0.192860,8


In [149]:
# Test that all occurences were counted correctly
speakers = dict()
for index, row in df.iterrows():
    speakers[index[0]] = speakers.get(index[0], 0) + 1

type(df)
df.dtypes
for i in range(1, 13):
    df = df.astype({f'coeff{i}': 'float32'})

df = df.astype({'speaker_count': 'int32'})
df.dtypes
print(res)
# df = df.sample(frac=1, random_state=42).reset_index(drop=True)

{0: 30, 1: 35, 2: 88, 3: 44, 4: 29, 5: 24, 6: 40, 7: 50, 8: 29, 9: 1}


In [150]:
df = df[~df.index.duplicated(keep='first')]
df.shape

(9961, 13)

In [151]:
colNames, windowed_df = new_dataframe()
past_utter = -1
past_frame = -1
count = 0

for index, row in df.iterrows():
    cur_utter = index[1]
    cur_frame = index[2]

    if cur_utter != past_utter:
        try:
            _, temp_df = new_dataframe()

            for i in range(WINDOW_SIZE):
                index = pd.MultiIndex.from_tuples([(int(count), int(i))])
                cur_row = pd.DataFrame(row.values.reshape(1, 13), index=index, columns=colNames)
                temp_df = pd.concat([temp_df, cur_row])

            windowed_df = pd.concat([windowed_df, temp_df])
            count += 1

        except KeyError:
            pass

    past_utter = cur_utter
    past_frame = cur_frame

In [152]:
windowed_df

,coeff1,coeff2,coeff3,coeff4,coeff5,coeff6,coeff7,coeff8,coeff9,coeff10,coeff11,coeff12,speaker_count
"(0, 0)",1.860936,-0.207383,0.261557,-0.214562,-0.171253,-0.118167,-0.277557,0.025668,0.126701,-0.306756,-0.213076,0.088728,0.0
"(0, 1)",1.860936,-0.207383,0.261557,-0.214562,-0.171253,-0.118167,-0.277557,0.025668,0.126701,-0.306756,-0.213076,0.088728,0.0
"(0, 2)",1.860936,-0.207383,0.261557,-0.214562,-0.171253,-0.118167,-0.277557,0.025668,0.126701,-0.306756,-0.213076,0.088728,0.0
"(0, 3)",1.860936,-0.207383,0.261557,-0.214562,-0.171253,-0.118167,-0.277557,0.025668,0.126701,-0.306756,-0.213076,0.088728,0.0
"(0, 4)",1.860936,-0.207383,0.261557,-0.214562,-0.171253,-0.118167,-0.277557,0.025668,0.126701,-0.306756,-0.213076,0.088728,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(639, 0)",1.421622,-0.386758,-0.187715,-0.504102,-0.026528,-0.56274,0.087821,0.168902,-0.137287,0.087146,-0.142142,0.101818,8.0
"(639, 1)",1.421622,-0.386758,-0.187715,-0.504102,-0.026528,-0.56274,0.087821,0.168902,-0.137287,0.087146,-0.142142,0.101818,8.0
"(639, 2)",1.421622,-0.386758,-0.187715,-0.504102,-0.026528,-0.56274,0.087821,0.168902,-0.137287,0.087146,-0.142142,0.101818,8.0
"(639, 3)",1.421622,-0.386758,-0.187715,-0.504102,-0.026528,-0.56274,0.087821,0.168902,-0.137287,0.087146,-0.142142,0.101818,8.0


In [153]:
speakers = dict()
for index, row in windowed_df.iterrows():
    speakers[index[0]] = speakers.get(index[0], 0) + 1

type(windowed_df)
print(windowed_df.dtypes)
for i in range(1, 13):
    windowed_df = windowed_df.astype({f'coeff{i}': 'float32'})

windowed_df = windowed_df.astype({'speaker_count': 'int32'})
print(windowed_df.dtypes)
print(res)

coeff1           object
coeff2           object
coeff3           object
coeff4           object
coeff5           object
coeff6           object
coeff7           object
coeff8           object
coeff9           object
coeff10          object
coeff11          object
coeff12          object
speaker_count    object
dtype: object
coeff1           float32
coeff2           float32
coeff3           float32
coeff4           float32
coeff5           float32
coeff6           float32
coeff7           float32
coeff8           float32
coeff9           float32
coeff10          float32
coeff11          float32
coeff12          float32
speaker_count      int32
dtype: object
{0: 30, 1: 35, 2: 88, 3: 44, 4: 29, 5: 24, 6: 40, 7: 50, 8: 29, 9: 1}


In [154]:
windowed_df

,coeff1,coeff2,coeff3,coeff4,coeff5,coeff6,coeff7,coeff8,coeff9,coeff10,coeff11,coeff12,speaker_count
"(0, 0)",1.860936,-0.207383,0.261557,-0.214562,-0.171253,-0.118167,-0.277557,0.025668,0.126701,-0.306756,-0.213076,0.088728,0
"(0, 1)",1.860936,-0.207383,0.261557,-0.214562,-0.171253,-0.118167,-0.277557,0.025668,0.126701,-0.306756,-0.213076,0.088728,0
"(0, 2)",1.860936,-0.207383,0.261557,-0.214562,-0.171253,-0.118167,-0.277557,0.025668,0.126701,-0.306756,-0.213076,0.088728,0
"(0, 3)",1.860936,-0.207383,0.261557,-0.214562,-0.171253,-0.118167,-0.277557,0.025668,0.126701,-0.306756,-0.213076,0.088728,0
"(0, 4)",1.860936,-0.207383,0.261557,-0.214562,-0.171253,-0.118167,-0.277557,0.025668,0.126701,-0.306756,-0.213076,0.088728,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(639, 0)",1.421622,-0.386758,-0.187715,-0.504102,-0.026528,-0.562740,0.087821,0.168902,-0.137287,0.087146,-0.142142,0.101818,8
"(639, 1)",1.421622,-0.386758,-0.187715,-0.504102,-0.026528,-0.562740,0.087821,0.168902,-0.137287,0.087146,-0.142142,0.101818,8
"(639, 2)",1.421622,-0.386758,-0.187715,-0.504102,-0.026528,-0.562740,0.087821,0.168902,-0.137287,0.087146,-0.142142,0.101818,8
"(639, 3)",1.421622,-0.386758,-0.187715,-0.504102,-0.026528,-0.562740,0.087821,0.168902,-0.137287,0.087146,-0.142142,0.101818,8


In [155]:
def splitData(dataset, split_fact):
    df1 = dataset.iloc[:int(split_fact[0]*len(dataset)),:]
    df2 = dataset.iloc[int(split_fact[0]*len(dataset)):,:]
    df3 = df2.iloc[int((split_fact[1]/(split_fact[1] + split_fact[2]))*len(df2)):,:]
    df2 = df2.iloc[:int((split_fact[1]/(split_fact[1] + split_fact[2]))*len(df2)),:]

    # t1 = df1.pop('speaker_count')
    # t2 = df2.pop('speaker_count')
    # t3 = df3.pop('speaker_count')

    # df1 = np.expand_dims(df1, 1)
    # df2 = np.expand_dims(df2, 1)
    # df3 = np.expand_dims(df3, 1)

    # return df1, t1, df2, t2, df3, t3
    return df1, df2, df3
# Convert df to DataSet

In [156]:
windowed_df.iloc[[2]].index

Index([(0, 2)], dtype='object')

In [157]:
def splitWindowData(df, split):
    _, train_df = new_dataframe()
    _, test_df = new_dataframe()

    train_idx = random.sample(range(0, 640), int(640*split[0]))

    for idx in train_idx:
        for i in range(WINDOW_SIZE):
            train_df = pd.concat([train_df, df.loc[[(idx, i)]]])
            df.drop([(idx, i)], inplace=True)

    test_idx = random.sample(range(0, 640 - int(640*split[0])), int((640 - int(640*split[0]))*split[1]))
    idx_list = []
    for idx in test_idx:
        for i in range(WINDOW_SIZE):
            test_df = pd.concat([test_df, df.iloc[[idx*WINDOW_SIZE + i]]])
            idx_list.append(df.iloc[[idx*WINDOW_SIZE + i]].index[0])

    for idx in idx_list:
        df.drop(idx, inplace=True)

    return train_df, test_df, df

In [158]:
train_x, test_x, val_x = splitWindowData(windowed_df, [0.7, 0.2, 0.1])
replaced_df = pd.concat([train_x, test_x, val_x])
replaced_df = replaced_df[~replaced_df.index.duplicated(keep='first')]

print(len(train_x), len(test_x), len(val_x))
print((len(train_x) + len(test_x) + len(val_x)), 3200, len(replaced_df))

2240 190 770
3200 3200 3200


In [159]:
train_x

,coeff1,coeff2,coeff3,coeff4,coeff5,coeff6,coeff7,coeff8,coeff9,coeff10,coeff11,coeff12,speaker_count
"(592, 0)",1.035833,-0.554279,0.076329,-0.698547,-0.075061,-0.259614,-0.170697,0.215351,-0.088709,-0.290493,0.02325,0.143952,7
"(592, 1)",1.035833,-0.554279,0.076329,-0.698547,-0.075061,-0.259614,-0.170697,0.215351,-0.088709,-0.290493,0.02325,0.143952,7
"(592, 2)",1.035833,-0.554279,0.076329,-0.698547,-0.075061,-0.259614,-0.170697,0.215351,-0.088709,-0.290493,0.02325,0.143952,7
"(592, 3)",1.035833,-0.554279,0.076329,-0.698547,-0.075061,-0.259614,-0.170697,0.215351,-0.088709,-0.290493,0.02325,0.143952,7
"(592, 4)",1.035833,-0.554279,0.076329,-0.698547,-0.075061,-0.259614,-0.170697,0.215351,-0.088709,-0.290493,0.02325,0.143952,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(166, 0)",1.18932,-0.522429,0.637606,-0.637568,-0.333684,-0.20493,0.194753,0.008195,-0.406871,-0.089503,-0.088683,-0.015278,5
"(166, 1)",1.18932,-0.522429,0.637606,-0.637568,-0.333684,-0.20493,0.194753,0.008195,-0.406871,-0.089503,-0.088683,-0.015278,5
"(166, 2)",1.18932,-0.522429,0.637606,-0.637568,-0.333684,-0.20493,0.194753,0.008195,-0.406871,-0.089503,-0.088683,-0.015278,5
"(166, 3)",1.18932,-0.522429,0.637606,-0.637568,-0.333684,-0.20493,0.194753,0.008195,-0.406871,-0.089503,-0.088683,-0.015278,5


In [160]:
test_x

,coeff1,coeff2,coeff3,coeff4,coeff5,coeff6,coeff7,coeff8,coeff9,coeff10,coeff11,coeff12,speaker_count
"(87, 0)",0.636909,0.124506,0.166842,-0.892269,0.139531,-0.775946,-0.099227,0.337095,-0.164165,0.052778,-0.126416,0.052365,2
"(87, 1)",0.636909,0.124506,0.166842,-0.892269,0.139531,-0.775946,-0.099227,0.337095,-0.164165,0.052778,-0.126416,0.052365,2
"(87, 2)",0.636909,0.124506,0.166842,-0.892269,0.139531,-0.775946,-0.099227,0.337095,-0.164165,0.052778,-0.126416,0.052365,2
"(87, 3)",0.636909,0.124506,0.166842,-0.892269,0.139531,-0.775946,-0.099227,0.337095,-0.164165,0.052778,-0.126416,0.052365,2
"(87, 4)",0.636909,0.124506,0.166842,-0.892269,0.139531,-0.775946,-0.099227,0.337095,-0.164165,0.052778,-0.126416,0.052365,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(558, 0)",0.987293,-0.084813,0.350135,-0.727016,0.081792,-0.168853,-0.266979,0.046818,0.19607,0.060369,-0.223176,0.060755,6
"(558, 1)",0.987293,-0.084813,0.350135,-0.727016,0.081792,-0.168853,-0.266979,0.046818,0.19607,0.060369,-0.223176,0.060755,6
"(558, 2)",0.987293,-0.084813,0.350135,-0.727016,0.081792,-0.168853,-0.266979,0.046818,0.19607,0.060369,-0.223176,0.060755,6
"(558, 3)",0.987293,-0.084813,0.350135,-0.727016,0.081792,-0.168853,-0.266979,0.046818,0.19607,0.060369,-0.223176,0.060755,6


In [161]:
val_x

,coeff1,coeff2,coeff3,coeff4,coeff5,coeff6,coeff7,coeff8,coeff9,coeff10,coeff11,coeff12,speaker_count
"(14, 0)",1.150639,-0.190578,0.562465,-0.146862,-0.402289,-0.385894,0.324414,-0.047720,-0.122438,-0.029703,-0.094095,0.158405,0
"(14, 1)",1.150639,-0.190578,0.562465,-0.146862,-0.402289,-0.385894,0.324414,-0.047720,-0.122438,-0.029703,-0.094095,0.158405,0
"(14, 2)",1.150639,-0.190578,0.562465,-0.146862,-0.402289,-0.385894,0.324414,-0.047720,-0.122438,-0.029703,-0.094095,0.158405,0
"(14, 3)",1.150639,-0.190578,0.562465,-0.146862,-0.402289,-0.385894,0.324414,-0.047720,-0.122438,-0.029703,-0.094095,0.158405,0
"(14, 4)",1.150639,-0.190578,0.562465,-0.146862,-0.402289,-0.385894,0.324414,-0.047720,-0.122438,-0.029703,-0.094095,0.158405,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
"(639, 0)",1.421622,-0.386758,-0.187715,-0.504102,-0.026528,-0.562740,0.087821,0.168902,-0.137287,0.087146,-0.142142,0.101818,8
"(639, 1)",1.421622,-0.386758,-0.187715,-0.504102,-0.026528,-0.562740,0.087821,0.168902,-0.137287,0.087146,-0.142142,0.101818,8
"(639, 2)",1.421622,-0.386758,-0.187715,-0.504102,-0.026528,-0.562740,0.087821,0.168902,-0.137287,0.087146,-0.142142,0.101818,8
"(639, 3)",1.421622,-0.386758,-0.187715,-0.504102,-0.026528,-0.562740,0.087821,0.168902,-0.137287,0.087146,-0.142142,0.101818,8


In [162]:
with open('Pickles/train_x.txt', 'wb') as tx:
    pickle.dump(train_x, tx)

# with open('Pickles/train_y.txt', 'wb') as ty:
#     pickle.dump(train_y, ty)

with open('Pickles/test_x.txt', 'wb') as tx:
    pickle.dump(test_x, tx)
#
# with open('Pickles/test_y.txt', 'wb') as ty:
#     pickle.dump(test_y, ty)

with open('Pickles/val_x.txt', 'wb') as vx:
    pickle.dump(val_x, vx)
#
# with open('Pickles/val_y.txt', 'wb') as vy:
#     pickle.dump(val_y, vy)
print("Done!")

Done!


In [163]:
for idx, row in train_x.iterrows():
    print(row.index)

Index(['coeff1', 'coeff2', 'coeff3', 'coeff4', 'coeff5', 'coeff6', 'coeff7',
       'coeff8', 'coeff9', 'coeff10', 'coeff11', 'coeff12', 'speaker_count'],
      dtype='object')
Index(['coeff1', 'coeff2', 'coeff3', 'coeff4', 'coeff5', 'coeff6', 'coeff7',
       'coeff8', 'coeff9', 'coeff10', 'coeff11', 'coeff12', 'speaker_count'],
      dtype='object')
Index(['coeff1', 'coeff2', 'coeff3', 'coeff4', 'coeff5', 'coeff6', 'coeff7',
       'coeff8', 'coeff9', 'coeff10', 'coeff11', 'coeff12', 'speaker_count'],
      dtype='object')
Index(['coeff1', 'coeff2', 'coeff3', 'coeff4', 'coeff5', 'coeff6', 'coeff7',
       'coeff8', 'coeff9', 'coeff10', 'coeff11', 'coeff12', 'speaker_count'],
      dtype='object')
Index(['coeff1', 'coeff2', 'coeff3', 'coeff4', 'coeff5', 'coeff6', 'coeff7',
       'coeff8', 'coeff9', 'coeff10', 'coeff11', 'coeff12', 'speaker_count'],
      dtype='object')
Index(['coeff1', 'coeff2', 'coeff3', 'coeff4', 'coeff5', 'coeff6', 'coeff7',
       'coeff8', 'coeff9', 'coeff10', 